# AAIT homework 2

## Familiarizing yourself with the data 

In [ ]:
# Preprocessing the data
# resizing the images, converting to a uniform format

## Choose a classifier 
begin implementing and testing your solution.

In [ ]:
# Download a pytorch MobileNet pretrained model

## Familiarizing with specific challenges 
missing labels and missing data

In [ ]:
# k-fold cross-validation, 
# ensemble methods, 
# or importance sampling

### Preprocessing the data according to challange of missing labels

# Solving the task 1

In [ ]:
# Fine tune the classifier
# try different parameters for the training: lr, regularization

# Test the model